# LFD Thursday Buffer File (Planning Pack) — SCM Day Prep

**Purpose**\
Provide sales/marketing and our LFD SCM team a consolidated, decision-ready view for the next 0–6 months forecast before **SCM Day**. Used to align revenue, sales, and production plans and to feed the weekly **PSI** (Production–Sales–Inventory) planning.

**Schedule**\
Auto-generated **Thursdays 07:00** (EST). Designed to run after HQ data refresh windows.

**Inputs (manual downloads)**
- **GSCM**: WOS_EX(RDD), B2B Risk, Arrival, EOH, RTF RAW, EOH
- **SAP**: ATS
- Weekly email sales data, Sellout Preliminary
- Product master **UPDATE** (specs, pricing, EOL, cusomters)

**What the notebook produces (by SKU × Account):**
- **AP1**, **RTF**, **calculated AP2**
- **WoW AP1** (week-over-week deltas)
- Product info (model, series, lifecycle status)
- Latest forecast & current-week **forecast guide**
- **Raw Weeks of Supply**, **current Ending On Hand (EOH)**
- **Adjusted sell-through** and **adjusted WOS**
- Reset **revenue / InTake RTF / PM AP2 / AP2 Buffer / WoW AP2** and **inventory allocation**
- Clean, formatted **Excel report** (emailed at 07:00 Thu)

**Process:** Ingest → clean/standardize → join product & price → compute KPIs (AP2, WoW, WOS, EOH, adjusted sell-through,etc.) → format → export report.

**Why it matters:**
- Replaces **2–3 hrs overtime** of early-morning manual work
- Runtime **< 20 minutes**; easy to use
- Improves **accuracy/consistency**

🤩READY FOR SCM DAY!🤩

Import packages and all necessary modules

In [1]:
import time
from tarfile import version

import openpyxl
from openpyxl import load_workbook
from openpyxl.descriptors import (String,Sequence,Integer)
from openpyxl.descriptors.serialisable import Serialisable
from openpyxl.styles import numbers,Alignment,PatternFill,Font,colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle
import numpy as np, pandas as pd
from pandas import DataFrame
import copy
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import warnings
import win32com.client as win32
import os
from sphinx.cmd.quickstart import nonempty
from sqlalchemy import false

Ignore warning for Python, no impact on performace

In [2]:
warnings.filterwarnings('ignore',category=UserWarning)

**Manuel Work Here**\
Input time and date for data searching and correct output 

In [3]:
today=input("what day is today? (yyyy,mm,dd)")
today=datetime.strptime(today,"%Y,%m,%d")
week=today.strftime("%V")
yr=today.strftime("%Y")
wk=yr+week
mon=today.strftime("%b")

what day is today? (yyyy,mm,dd) 2025,04,08


Define all functions, including extracting data from GSCM and SAP, cleanning data, formatting, calculation and output\
DO NOT EDIT

In [7]:
def clean(ws,maxcol,sc,sr):
    for col in ws.iter_cols(min_col=sc, max_col=maxcol,min_row=sr, max_row=len(ws['D'])):
        for cell in col:
            cell.value=None
            cell.font = NamedStyle().font
            cell.fill = NamedStyle().fill
            cell.border = NamedStyle().border
            cell.alignment = NamedStyle().alignment
            cell.number_format = NamedStyle().number_format

In [9]:
# Identify different sheet in Buffer file and apply different calculations and format
# Not like the direct final result, I have to write down the formula into the excel sheet for MKT review
# clean pivot table if needed

def gscm(target, gfile, tsheet):
    with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
        workbook = writer.book
        w1 = writer.sheets[tsheet]
        w2_wb = load_workbook(gfile)
        w2 = w2_wb.active
        
        if tsheet =='EOH Raw':
            found_maxcol = 0
            for cols in w1.iter_cols(min_row=1,max_row=1):
                for cell in cols:
                    if cell.value == "Total":
                        maxcol=cell.column
                        found_maxcol = 1
                if found_maxcol == 1:
                    break
            clean(w1, maxcol,1,1)
            
        elif tsheet=='Raw':
            found_maxcol = 0
            for cols in w1.iter_cols(min_row=2,max_row=2):
                for cell in cols:
                    if cell.value == "Total":
                        maxcol=cell.column
                        found_maxcol = 1
                if found_maxcol == 1:
                    break
            clean(w1, maxcol,4,2)
        else:
            found_maxcol = 0
            for cols in w1.iter_cols(min_row=2,max_row=2):
                for cell in cols:
                    if cell.value == "Total":
                        maxcol=cell.column
                        found_maxcol = 1
                if found_maxcol == 1:
                    break
            clean(w1, maxcol,1,2)
                    
        if tsheet=='EOH Raw':
            for row in w2.iter_rows(min_row=2, min_col=2):
                for cell in row:
                    nc = w1.cell(row=cell.row - 1, column=cell.column - 1, value=cell.value)
                    if cell.has_style:
                        nc.font = copy.copy(cell.font)
                        nc.fill = copy.copy(cell.fill)
                        nc.border = copy.copy(cell.border)
                        nc.alignment = copy.copy(cell.alignment)
                        nc.number_format = copy.copy(cell.number_format)
    
            eoh_sheet=writer.sheets['EOH']
            pivot = eoh_sheet._pivots[0]
            pivot.source = "'EOH Raw'!A1:{}".format(nc.coordinate)
            pivot.cache.refreshOnLoad = True
    
            found_eohrow = 0
            for rows in eoh_sheet.iter_rows(min_col=2,max_col=2):
                for cell in rows:
                    if cell.value == "Grand Total":
                        maxrow=cell.row
                        found_eohrow = 1
                if found_eohrow == 1:
                    break
            for i in range(1,maxrow+1):
                sku=eoh_sheet[f'B{i}'].value
                eoh_sheet[f'R{i}'].value=sku
                
        elif tsheet=='Raw':
            
            for row in w2.iter_rows(min_row=2, min_col=2):
                for cell in row:
                    nc = w1.cell(row=cell.row, column=cell.column + 2, value=cell.value)
                    if cell.has_style:
                        nc.font = copy.copy(cell.font)
                        nc.fill = copy.copy(cell.fill)
                        nc.border = copy.copy(cell.border)
                        nc.alignment = copy.copy(cell.alignment)
                        nc.number_format = copy.copy(cell.number_format)
    
            found_maxrow = 0
            for rows in w1.iter_rows(min_row=200, min_col=5, max_col=5):
                for cell in rows:
                    if cell.value is None:
                        maxrow = cell.row
                        found_maxrow = 1
                if found_maxrow == 1:
                    break
            if found_maxrow == 0:
                maxrow = cell.row + 1
                
            mon_idx=0
            wk_idx=0
            nondate_idx=0
            for toprow in w1.iter_rows(min_row=2, max_row=2):
                    for header in toprow:
                        if header.value == "Category":
                            nondate_idx = header.col_idx
                        elif header.value == wk:
                            wk_idx = header.col_idx
                        elif header.value == mon:
                            mon_idx = header.col_idx
                        elif wk_idx != 0 and nondate_idx != 0 and mon_idx != 0:
                            break
            wk_qty = mon_idx - wk_idx
            if wk_idx-nondate_idx == 1:
                gi=0
            elif wk_idx-nondate_idx == 2:
                gi=1
            elif wk_idx-nondate_idx > 2:
                gi=2
    
            for i in range(11,maxrow):
                j = 0
                total = 0
                if (i-10) % 8 == 1:
                    if gi==0:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i)
                    else:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+6,get_column_letter(wk_idx-1),i+6)
                elif (i-10) % 8 == 2:
                    if gi<2:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(wk_idx-1),i,get_column_letter(mon_idx-1),i)
                    else:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{})".format(get_column_letter(wk_idx-1),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+5,get_column_letter(wk_idx-2),i+5)
                elif (i-10) % 8 == 3:
                    if gi == 0:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{})".format(get_column_letter(wk_idx+1),i,get_column_letter(mon_idx-1),i,get_column_letter(wk_idx),i+2)
                    elif gi > 0 and wk_qty>1:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{},{}{})".format(get_column_letter(wk_idx+1),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+4,get_column_letter(wk_idx-1),i+4,get_column_letter(wk_idx),i+2)
                    elif wk_qty==1:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{})".format(get_column_letter(nondate_idx+1),i+4,get_column_letter(wk_idx-1),i+4,get_column_letter(wk_idx),i+2)
                elif (i-10) % 8 == 4:
                    if gi == 0:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i)
                    elif gi == 1:                                                     
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i,get_column_letter(wk_idx-1),i+1)
                    else:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{},{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+3,get_column_letter(wk_idx-2),i+3,get_column_letter(wk_idx-1),i+1)
                elif (i-10) % 8 == 5:
                    if gi==0:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i)
                    else:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{})".format(get_column_letter(wk_idx),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+2,get_column_letter(wk_idx-1),i+2)
                elif (i-10) % 8 == 6:
                    if gi<2:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(wk_idx-1),i,get_column_letter(mon_idx-1),i)
                    else:
                        w1.cell(i, mon_idx).value = "=SUM({}{}:{}{},{}{}:{}{})".format(get_column_letter(wk_idx-1),i,get_column_letter(mon_idx-1),i,get_column_letter(nondate_idx+1),i+1,get_column_letter(wk_idx-2),i+1)
                elif (i-10) % 8 == 7:
                    w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(nondate_idx+1),i,get_column_letter(mon_idx-1),i)
                elif (i-10) % 8 == 0:
                    w1.cell(i, mon_idx).value = "=SUM({}{}:{}{})".format(get_column_letter(nondate_idx+1),i,get_column_letter(mon_idx-1),i) 
        else:
            for row in w2.iter_rows(min_row=2, min_col=2):
                for cell in row:
                    nc = w1.cell(row=cell.row, column=cell.column - 1, value=cell.value)
                    if cell.has_style:
                        nc.font = copy.copy(cell.font)
                        nc.fill = copy.copy(cell.fill)
                        nc.border = copy.copy(cell.border)
                        nc.alignment = copy.copy(cell.alignment)
                        nc.number_format = copy.copy(cell.number_format)


In [11]:
def sap(target, sfile):
    with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
        workbook = writer.book
        w1 = writer.sheets["SAP ATS"]
        w2_wb = load_workbook(sfile)
        w2 = w2_wb.active

        clean(w1, 15,1,1)
        for row in w2.iter_rows(min_row=5, min_col=1):
            for cell in row:
                nc = w1.cell(row=cell.row-4, column=cell.column, value=cell.value)
                if cell.has_style:
                    nc.font = copy.copy(cell.font)
                    nc.fill = copy.copy(cell.fill)
                    nc.border = copy.copy(cell.border)
                    nc.alignment = copy.copy(cell.alignment)
                    nc.number_format = copy.copy(cell.number_format)

        pivot = w1._pivots[0]
        pivot.cache.refreshOnLoad = True

        for cells in w1.iter_rows(min_row=100, min_col=17, max_col=17):
            for cell in cells:
                if cell.value=="Grand Total":
                    total_row=cell.row
        for i in range(100,total_row+1):
            w1.cell(i, 21).value = "=R{}-S{}-T{}".format(i,i,i)

For sellthrough or other sales data, please check the file first.\
The preliminary file is the only file that I standardize into Python. Others are **not** required, and are **only** needed as needed if MKT asked.

In [13]:
def pre(target, preli):
    df=pd.read_excel(preli,sheet_name=1,usecols=[0,1,2,3,17,28],engine='openpyxl')
    df=df[(df['Unnamed: 0']=='B2B') & (df['Unnamed: 1']=='LFD MON')]
    df=df.reset_index(drop=True)
    with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer: 
        workbook = writer.book
        del workbook['Sellout Prelim']
        workbook.create_sheet(title='Sellout Prelim',index=19)
        ws=writer.sheets['Sellout Prelim']
        ws.sheet_properties.tabColor = '00CCFFCC'
        ws.append(list(df.columns))
        title=('B2B B2C cust','Material Group','Material','Pet Name','PC','')
        ws.append(title)
        for row in df.itertuples(index=False):
            ws.append(row)

**Manule Work**\
Write down the file path below and click run.

In [15]:
target = "C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\WK15 AP2 Buffer File LFD_04092025.xlsx"
rdd= "C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\rdd.xlsx"
b2b="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\b2b.xlsx"
raw="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\raw.xlsx"
arr="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\arr.xlsx"
eoh="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\eoh.xlsx"
ats="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\ats.xlsx"
preli="C:\\Users\\zhuoyan.bai\\Documents\\buffer file\\wk15\\W15 Display Sellout Preliminary Report.xlsx"

The next cell is for me to track the Python performance and find opportunity to improve, skip this cell if you want.

In [17]:
# AP1 change in the buffer file
t0=time.time()
gscm(target, rdd, 'WOS_EX(RDD)')
t1=time.time()
gscm(target, b2b, 'B2B Risk')
t2=time.time()
gscm(target, raw, 'Raw')
t3=time.time()
gscm(target, arr, 'Next Intake RTF')
t4=time.time()
gscm(target, eoh, 'EOH Raw')
t5=time.time()
sap(target, ats)
t6=time.time()
pre(target, preli)
t7=time.time()
print("rdd:",t1-t0,"b2b:",t2-t1,"raw:",t3-t2,"arr:",t4-t3,"eoh:",t5-t4,"ats:",t6-t5,"pre:",t7-t6)   

# rdd: 56.75175404548645 b2b: 70.58072662353516 raw: 92.72838973999023 arr: 67.68977355957031 eoh: 74.73189640045166 ats: 90.16780948638916 pre: 97.91332507133484

rdd: 82.51137232780457 b2b: 93.87524485588074 raw: 131.25304913520813 arr: 132.93405175209045 eoh: 140.8801884651184 ats: 140.95684814453125 pre: 161.69107723236084


**Final Step**\
Write into the buffer file, and ready to send out the email after finish.

In [18]:
with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer: 
    workbook = writer.book
    ws=writer.sheets["Summary"]
    found_maxrow = 0
    for rows in ws.iter_rows(min_row=850,min_col=4, max_col=4):
        for cell in rows:
            if cell.value is None:
                maxrow = cell.row
                found_maxrow = 1
        if found_maxrow == 1:
            break
    if found_maxrow == 0:
        maxrow = cell.row + 1
    for i in range (28,maxrow):
        ws.cell(i,10).value = "=IFERROR(INDEX(BOB!$A:$AP,MATCH(Summary!$D{}&Summary!$J$5,BOB!$A:$A,0),MATCH(Summary!$E{},BOB!$1:$1,0)),0)".format(i,i)
        ws.cell(i,11).value = "=IFERROR(INDEX('Next Intake RTF'!$A:$AP,MATCH(D{},'Next Intake RTF'!$A:$A,0),MATCH(E{},'Next Intake RTF'!$1:$1,0)),0)".format(i,i)
        ws.cell(i,12).value = "=R{}".format(i)
        

In [1]:
#%config completer.use_jedi=False